In [22]:
import requests
from google.colab import files
import json
import pandas as pd
import numpy as np

In [23]:
load = files.upload()
get = next(iter(load.values()))
secret = json.loads(get.decode())

Saving spotify-hidden.json to spotify-hidden (1).json


In [24]:
clientid = secret["client_id"]
clientsecret = secret["client_secret"]
accesstoken = secret["access_token"]

In [25]:
auth_url = 'https://accounts.spotify.com/api/token'

auth_response = requests.post(auth_url, {
    'grant_type': 'client_credentials',
    'client_id': clientid,
    'client_secret': clientsecret,
    'access_token': accesstoken
})

auth_response_info = auth_response.json()

access_token = auth_response_info['access_token']

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

#for GET reqeuests to API

generic_url = 'https://api.spotify.com/v1/'


In [26]:
id = input("Enter seed playlist link ")
start = id.find('/playlist/')
id = id[start+10:id.find('?', start)]

Enter seed playlist link https://open.spotify.com/playlist/0CS87zvUY3sKVUFh6tPrmI?si=27e16b0c85a84159


In [27]:
id_two = input("Enter playlist link that you want to extract songs from ")

id_two = id_two[start+10:id_two.find('?', start)]

Enter playlist link that you want to extract songs from https://open.spotify.com/playlist/5KWjJCREgsPlUiHVLuYvfg?si=596673de228046c1


In [28]:
def create_list_URI(id):
  playlist = requests.get(generic_url+'playlists/' + id + '/tracks', headers = headers).json()
  track_uri = []
  if 'items' in playlist:
    for item in playlist['items']:
      track_uri.append(item['track']['uri'][-22:])
  else:
    print("Playlists must be public")

  return track_uri

# get tracks from playlist and extract trackURI

In [29]:
id1_urilist = create_list_URI(id)

In [30]:
id2_urilist = create_list_URI(id_two)

In [31]:
def get_song_desc(list_of_trackURIs):
  song_desc = []
  for item in list_of_trackURIs:
    song_desc.append(requests.get(generic_url+'audio-features/' + item, headers = headers).json())

  return song_desc


# creates a list of dictionaries (dicts are the result of running the trackURI through spotify to get detailed song info)
# info on metrics https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features

In [32]:
id1_track_info = get_song_desc(id1_urilist)

#id1_track_info
seed_df = pd.json_normalize(id1_track_info)

seed_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.878,0.525,5,-6.832,1,0.0578,0.1830,0.000000,0.0714,0.694,112.022,audio_features,1Cv1YLb4q0RzL6pybtaMLo,spotify:track:1Cv1YLb4q0RzL6pybtaMLo,https://api.spotify.com/v1/tracks/1Cv1YLb4q0Rz...,https://api.spotify.com/v1/audio-analysis/1Cv1...,158571,4
1,0.658,0.294,4,-8.533,0,0.0321,0.4220,0.000052,0.0749,0.358,74.038,audio_features,7zFXmv6vqI4qOt4yGf3jYZ,spotify:track:7zFXmv6vqI4qOt4yGf3jYZ,https://api.spotify.com/v1/tracks/7zFXmv6vqI4q...,https://api.spotify.com/v1/audio-analysis/7zFX...,278180,4
2,0.558,0.559,6,-9.222,1,0.0959,0.3710,0.000007,0.1090,0.620,78.558,audio_features,7KA4W4McWYRpgf0fWsJZWB,spotify:track:7KA4W4McWYRpgf0fWsJZWB,https://api.spotify.com/v1/tracks/7KA4W4McWYRp...,https://api.spotify.com/v1/audio-analysis/7KA4...,180387,4
3,0.756,0.609,3,-7.927,1,0.0544,0.2170,0.004820,0.0959,0.537,117.024,audio_features,5EYi2rH4LYs6M21ZLOyQTx,spotify:track:5EYi2rH4LYs6M21ZLOyQTx,https://api.spotify.com/v1/tracks/5EYi2rH4LYs6...,https://api.spotify.com/v1/audio-analysis/5EYi...,155720,4
4,0.698,0.633,8,-4.946,1,0.0597,0.0605,0.000194,0.0926,0.552,96.924,audio_features,3GCdLUSnKSMJhs4Tj6CV3s,spotify:track:3GCdLUSnKSMJhs4Tj6CV3s,https://api.spotify.com/v1/tracks/3GCdLUSnKSMJ...,https://api.spotify.com/v1/audio-analysis/3GCd...,232187,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,0.726,0.431,8,-8.765,0,0.1350,0.7310,0.000000,0.6960,0.348,144.026,audio_features,7eJMfftS33KTjuF7lTsMCx,spotify:track:7eJMfftS33KTjuF7lTsMCx,https://api.spotify.com/v1/tracks/7eJMfftS33KT...,https://api.spotify.com/v1/audio-analysis/7eJM...,173333,4
81,0.752,0.650,4,-6.194,1,0.0334,0.2640,0.000001,0.3130,0.904,141.160,audio_features,0ClnfM4PI0wjlCVTJc9lP7,spotify:track:0ClnfM4PI0wjlCVTJc9lP7,https://api.spotify.com/v1/tracks/0ClnfM4PI0wj...,https://api.spotify.com/v1/audio-analysis/0Cln...,177000,4
82,0.733,0.653,11,-6.094,1,0.0391,0.0179,0.021800,0.1840,0.752,119.996,audio_features,1QxIFMs4M8qd5cbJJu80v4,spotify:track:1QxIFMs4M8qd5cbJJu80v4,https://api.spotify.com/v1/tracks/1QxIFMs4M8qd...,https://api.spotify.com/v1/audio-analysis/1QxI...,182895,5
83,0.843,0.552,6,-6.459,0,0.0619,0.4780,0.000964,0.1000,0.740,110.019,audio_features,1ZUBzB8QOM6a6V3gwuPeFL,spotify:track:1ZUBzB8QOM6a6V3gwuPeFL,https://api.spotify.com/v1/tracks/1ZUBzB8QOM6a...,https://api.spotify.com/v1/audio-analysis/1ZUB...,201308,4


In [33]:
id2_track_info = get_song_desc(id2_urilist)

#id2_track_info


In [34]:
#getting average stats of tracks from seed playlist (for vector)

means = seed_df.mean(axis=0)
# Create a new DataFrame with the mean values (and only important metrics)
mean_df = pd.DataFrame(means).T
mean_df.drop('mode', axis=1, inplace=True)
mean_df.drop('duration_ms', axis=1, inplace=True)
mean_df.drop('time_signature', axis=1, inplace=True)
mean_df.drop('key', axis=1, inplace=True)

mean_df

<ipython-input-34-05d525a197ef>:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  means = seed_df.mean(axis=0)


,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.688153,0.604553,-6.2896,0.090925,0.285299,0.00108,0.181839,0.553738,111.616671


In [35]:
# convert to array for distance comparison
vector = np.array(mean_df.loc[0, :].values.tolist())

In [36]:
#convert the second playlist to df

second_df = pd.json_normalize(id2_track_info)

#remove the unnecessary columns
second_df.drop('mode', axis=1, inplace=True)
second_df.drop('key', axis=1, inplace=True)
second_df.drop('type', axis=1, inplace=True)
second_df.drop('uri', axis=1, inplace=True)
second_df.drop('track_href', axis=1, inplace=True)
second_df.drop('analysis_url', axis=1, inplace=True)
second_df.drop('duration_ms', axis=1, inplace=True)
second_df.drop('time_signature', axis=1, inplace=True)
if 'error.status' in second_df:
  second_df = second_df.drop('error.status', axis=1)
if 'error.message' in second_df:
  second_df = second_df.drop('error.message', axis=1)

second_df


,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id
0,0.734,0.874,-3.158,0.0662,0.1680,0.000011,0.0489,0.905,122.953,6cx06DFPPHchuUAcTxznu9
1,0.674,0.774,-7.567,0.0892,0.0686,0.001880,0.3930,0.330,112.050,62aP9fBQKYKxi7PDXwcUAS
2,0.632,0.595,-7.644,0.0401,0.0689,0.000000,0.2090,0.435,124.114,6ft4hAq6yde8jPZY2i5zLr
3,0.659,0.617,-5.862,0.0649,0.2700,0.000000,0.1870,0.187,119.858,5ow0sNF1zSqp71Ix5jEXWU
4,0.880,0.751,-4.258,0.0874,0.1770,0.000064,0.1060,0.884,117.948,2tnVG71enUj33Ic2nFN6kZ
...,...,...,...,...,...,...,...,...,...,...
95,0.789,0.720,-5.652,0.2180,0.2230,0.000000,0.1290,0.664,122.031,3H7ihDc1dqLriiWXwsc2po
96,0.704,0.899,-5.610,0.1780,0.1890,0.000024,0.2280,0.576,123.028,7sFk2PA3NsrrSsGa4CM3rs
97,0.595,0.846,-4.806,0.1460,0.0842,0.001160,0.0689,0.327,123.215,4xH2aj3tn2PF9qq7YlrTux
98,0.715,0.635,-6.591,0.0791,0.1270,0.000000,0.0862,0.390,112.887,5Gj1wG8b12VQdEd3hUuSwo


In [37]:
#create a dictionary with the URI as the key to the array of integers (value)
dictionary = second_df.set_index('id').T.to_dict('list')

for key, value in dictionary.items():
  dictionary[key] = np.array(value)


#compute distance from average (seed vector) to each song
def comparison(dictionary, vector):
  new = []
  for key in dictionary:
    new.append((key, np.linalg.norm(vector - dictionary[key])))
  
  return new

<ipython-input-37-359363576cea>:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dictionary = second_df.set_index('id').T.to_dict('list')


In [ ]:
sorted_data = sorted(comparison(dictionary, vector), key=lambda x: x[1])
#sorted list of tuples with URI + distance (by closest to furthest)

#sorted_data

In [39]:
num = int(input("How many songs do you want?. Maximum is " + str(len(sorted_data))) + " ")

sublist = sorted_data[:num]
sublist

How many songs do you want?. Maximum is 9915


[('5Gj1wG8b12VQdEd3hUuSwo', 1.3294295983300624),
 ('62aP9fBQKYKxi7PDXwcUAS', 1.4106904614712226),
 ('0ZUE5EIBOy95Abo533ltQN', 1.663662637530125),
 ('5kmGLWKHzGIRSR8uBRxvNg', 2.0543600053024904),
 ('3iBTBR4dCnQAtZM5GJHqDD', 2.679537088691747),
 ('6Y2kDIq8xmo0sYZE3o7KCE', 3.3370374368734947),
 ('6LwDwlxumqnruc7Wl02RO1', 3.3825000578720443),
 ('2ycqUyUXI9oZVThwtGCkWd', 3.468286997207216),
 ('3sJo354zgoxfn6GH8hFBLm', 3.617174818536826),
 ('4oFB5gXKF5TLs3yW5DGdHi', 4.036324397341487),
 ('7BTXeU5C7IkvvEuw2HcmDr', 4.044739285014454),
 ('0N8nBk5KB98FYknkyOLjll', 4.44987613425317),
 ('4REVgcl64gB7g5U8pI7Z4U', 4.7259526686068085),
 ('1kBG0zsdOoyPeNRiCXgwh4', 5.540974567432925),
 ('2UbVnbE5FH6008mAm6Mmgw', 5.709021168454137)]

In [40]:
final_list = []
sub_final_list = []
for item in sublist:
  sub_final_list.append(requests.get(generic_url+'tracks/' + item[0], headers = headers).json())

#get list of songs (and all track info) (note API call after user requests how many songs they want)

for song in sub_final_list:
  final_list.append((song['name'],song['external_urls']['spotify']))

#filter to only the name of the song and the link to the song

final_list

[('Freedom', 'https://open.spotify.com/track/5Gj1wG8b12VQdEd3hUuSwo'),
 ('ily (i love you baby) (feat. Emilee)',
  'https://open.spotify.com/track/62aP9fBQKYKxi7PDXwcUAS'),
 ('Sky Comes Down', 'https://open.spotify.com/track/0ZUE5EIBOy95Abo533ltQN'),
 ('I Wanna Love You', 'https://open.spotify.com/track/5kmGLWKHzGIRSR8uBRxvNg'),
 ('Back to You', 'https://open.spotify.com/track/3iBTBR4dCnQAtZM5GJHqDD'),
 ('Pick Me Up', 'https://open.spotify.com/track/6Y2kDIq8xmo0sYZE3o7KCE'),
 ('Into You', 'https://open.spotify.com/track/6LwDwlxumqnruc7Wl02RO1'),
 ('Never Enough', 'https://open.spotify.com/track/2ycqUyUXI9oZVThwtGCkWd'),
 ('Expectations (R3HAB Remix)',
  'https://open.spotify.com/track/3sJo354zgoxfn6GH8hFBLm'),
 ("Don't Know Me", 'https://open.spotify.com/track/4oFB5gXKF5TLs3yW5DGdHi'),
 ('Be Okay', 'https://open.spotify.com/track/7BTXeU5C7IkvvEuw2HcmDr'),
 ('Way Out', 'https://open.spotify.com/track/0N8nBk5KB98FYknkyOLjll'),
 ('Wonderwall', 'https://open.spotify.com/track/4REVgcl64gB7g